In [1]:
# hide
# default_exp core.compose
import os
from nbdev.showdoc import *
if not os.path.exists('settings.ini'):
    os.chdir('..')

# Compose components

> Classes and utilities for composed components

In [2]:
#export
import warnings
import pandas as pd
import warnings
from sklearn.utils import Bunch

from block_types.core.block_types import Component, PandasComponent, SamplingComponent
from block_types.core.data_conversion import PandasConverter
from block_types.core.utils import PandasIO

## MultiComponent

In [3]:
#export
class MultiComponent (SamplingComponent):
    """
    Component containing a list of components inside.
    
    The list must contain at least one component. 
    
    See `Pipeline` class.
    """
    def __init__ (self, separate_labels = False, **kwargs):
        """Assigns attributes and calls parent constructor.

        Parameters
        ----------
        separate_labels: bool, optional
            whether or not the fit method receives the labels in a separate `y` vector 
            or in the same input `X`, as an additional variable. See description of 
            Pipeline class for more details.
        """
        if not hasattr (self, 'components'):
            self.components = []
        if not hasattr (self, 'finalized_component_list'):
            self.finalized_component_list = False
        
        # we need to call super().__init__() *after* having creating the `components` field,
        # since the constructor of Component calls a method that is overriden in Pipeline, 
        # and this method makes use of the mentioned `components` field
        super().__init__ (separate_labels = separate_labels, 
                          **kwargs)

        self.set_split ('whole')
        
    
    def register_components (self, *components):
        """
        Registering component in `self.components` list.
        
        Every time that a new component is set as an attribute of the pipeline,
        this component is added to the list `self.components`. Same 
        mechanism as the one used by pytorch's `nn.Module`
        """
        if not hasattr(self, 'components'):
            self.components = []
            self.finalized_component_list = False
        if not self.finalized_component_list:
            self.components += components
    
    def __setattr__(self, k, v):
        """
        See register_components
        """
        super().__setattr__(k, v)
        
        if isinstance(v, Component):
            self.register_components(v)
            if hasattr(v, 'nick_name'):
                self.logger.warning (f'{v} already has a nick_name: {v.nick_name}')
                warnings.warn (f'{v} already has a nick_name: {v.nick_name}')
            v.nick_name = k
            
    def add_component (self, component):
        if not hasattr(self, 'finalized_component_list'):
            self.finalized_component_list = False
        finalized_component_list = self.finalized_component_list
        self.finalized_component_list = False
        self.register_components(component)
        self.finalized_component_list = finalized_component_list
        
        if hasattr(component, 'nick_name'):
            self.logger.warning (f'{component} already has a nick_name: {component.nick_name}')
            warnings.warn (f'{component} already has a nick_name: {component.nick_name}')
        component.nick_name = component.name
        if not hasattr(self, component.name):
            self.__setattr__ (component.name, component)
        
    def set_components (self, *components):
        self.components = components
        self.finalized_component_list = True
        for component in components:
            if hasattr(component, 'nick_name'):
                self.logger.warning (f'{component} already has a nick_name: {component.nick_name}')
                warnings.warn (f'{component} already has a nick_name: {component.nick_name}')
            component.nick_name = component.name
            if not hasattr(self, component.name):
                self.__setattr__ (component.name, component)
        
    def clear_descendants (self):
        self.cls = Bunch ()
        self.obj = Bunch ()
        self.full_obj = Bunch ()
        self.full_cls = Bunch ()
        for component in self.components:
            if isinstance(component, MultiComponent):
                component.clear_descendants ()
            
    def gather_descendants (self, root='', nick_name=True):
        if not hasattr (self, 'cls'):
            self.cls = Bunch ()
            self.obj = Bunch ()
            self.full_obj = Bunch ()
            self.full_cls = Bunch ()
            
        if hasattr(self, 'nick_name'):
            name = self.nick_name if nick_name else self.name
        else:
            name = self.name
        self.hierarchy_path = f'{root}{name}'
        for component in self.components:
            self._insert_descendant (self.cls, component, component.class_name)
            self._insert_descendant (self.obj, component, component.name)
            
            name = component.nick_name if nick_name else component.name
            component_hierarchy_path = f'{self.hierarchy_path}.{name}'
            self._insert_descendant (self.full_cls, component_hierarchy_path, component.class_name)
            self._insert_descendant (self.full_obj, component_hierarchy_path, component.name)
            if isinstance(component, MultiComponent):
                component.gather_descendants (root=f'{self.hierarchy_path}.',
                                              nick_name=nick_name)
                for name in component.cls:
                    self._insert_descendant (self.cls, component.cls[name], name)
                    self._insert_descendant (self.full_cls, component.full_cls[name], name)
                for name in component.obj:
                    self._insert_descendant (self.obj, component.obj[name], name)
                    self._insert_descendant (self.full_obj, component.full_obj[name], name)
                           
    def _insert_descendant (self, cmp_dict, component, name):
        if name in cmp_dict:
            if not isinstance(cmp_dict[name], list):
                cmp_dict[name] = [cmp_dict[name]]
            if isinstance(component, list):
                cmp_dict[name].extend(component)
            else:
                cmp_dict[name].append(component)
        else:
            if isinstance(component, list):
                cmp_dict[name] = component.copy()
            else:
                cmp_dict[name] = component
        
    def construct_diagram (self, split=None, include_url=False, port=4000, project='block_types'):
        """
        Construct diagram of the pipeline components, data flow and dimensionality.
        
        By default, we use test data to show the number of observations 
        in the output of each component. This can be changed passing 
        `split='train'`
        """
        split = self.get_split (split)

        if include_url:
            base_url = f'http://localhost:{port}/{project}'
        else:
            URL = ''

        node_name = 'data'
        output = 'train / test'

        f = Digraph('G', filename='fsm2.svg')
        f.attr('node', shape='circle')

        f.node(node_name)

        f.attr('node', shape='box')
        for component in self.components:
            last_node_name = node_name
            last_output = output
            node_name = component.model_plotter.get_node_name()
            if include_url:
                URL = f'{base_url}/{component.model_plotter.get_module_path()}.html#{node_name}'
            f.node(node_name, URL=URL)
            f.edge(last_node_name, node_name, label=last_output)
            output = component.model_plotter.get_edge_name(split=split)

        last_node_name = node_name
        node_name = 'output'
        f.attr('node', shape='circle')
        f.edge(last_node_name, node_name, label=output)

        return f

    def show_result_statistics (self, split=None):
        """
        Show statistics about results obtained by each component. 
        
        By default, this is shown on test data, although this can change setting 
        `split='train'`
        """
        split = self.get_split (split)

        for component in self.components:
            component.show_result_statistics(split=split)

    def show_summary (self, split=None):
        """
        Show list of pipeline components, data flow and dimensionality.
        
        By default, we use test data to show the number of observations 
        in the output of each component. This can be changed passing 
        `split='train'`
        """
        split = self.get_split (split)

        node_name = 'data'
        output = 'train / test'

        for i, component in enumerate(self.components):
            node_name = component.model_plotter.get_node_name()
            output = component.model_plotter.get_edge_name(split=split)
            print (f'{"-"*100}')
            print (f'{i}: {node_name} => {output}')


    def get_split (self, split=None):
        if split is None:
            if self.data_io.split is not None:
                split = self.data_io.split
            else:
                split = 'whole'

        return split

    def assert_equal (self, path_reference_results, assert_equal_func=pd.testing.assert_frame_equal, **kwargs):
        """Compare results stored in current run against reference results stored in given path."""

        for component in self.components:
            component.assert_equal (path_reference_results, assert_equal_func=assert_equal_func, **kwargs)
        self.logger.info ('both pipelines give the same results')
        print ('both pipelines give the same results')
        
    def load_estimator (self):
        for component in self.components:
            component.load_estimator ()

    # *************************
    # setters
    # *************************
    def set_split (self, split):
        super().set_split (split)
        for component in self.components:
            component.set_split (split)

    def set_save_splits (self, save_splits):
        super().set_save_splits (save_splits)
        for component in self.components:
            component.set_save_splits (save_splits)
            
    def set_load_model (self, load_model):
        super().set_load_model (load_model)
        for component in self.components:
            component.set_load_model (load_model)

    def set_save_model (self, save_model):
        super().set_save_model (save_model)
        for component in self.components:
            component.set_save_model (save_model)
            
    def set_save_result (self, save_result):
        super().set_save_result (save_result)
        for component in self.components:
            component.set_save_result (save_result)
            
    def set_load_result (self, load_result):
        super().set_load_result (load_result)
        for component in self.components:
            component.set_load_result (load_result)

### Usage examples / tests

#### Loading / saving all results

In [4]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer
from block_types.core.utils import PickleIO
from block_types.utils.utils import remove_previous_results

path_results = 'multi_component_loading_saving'
remove_previous_results (path_results=path_results)

# 1. by setting components as attributes:
class NewComposition(MultiComponent):
    
    def __init__ (self, **kwargs):
        data_io = PickleIO (**kwargs)
        super().__init__(data_io=data_io,
                         **kwargs)
        
        self.tr1 = Component(FunctionTransformer (lambda x: x*3),
                             data_io=data_io,
                             name='tr1')
        self.tr2 = Component(FunctionTransformer (lambda x: x*2),
                             data_io=data_io,
                             name='tr2')
        
    def _apply (self, X):
        return self.tr1 (X) + self.tr2(X)

X = np.array([1,2,3])
    
composition1 = NewComposition (path_results=path_results)
result1 = composition1 (X)

composition2 = NewComposition (path_results=path_results)
result2 = composition2.data_io.load_result()
assert (result1==result2).all()

resut_tr1_2 = composition2.tr1.data_io.load_result()
resut_tr2_2 = composition2.tr2.data_io.load_result()
assert (resut_tr1_2==composition1.tr1(X)).all()
assert (resut_tr2_2==composition1.tr2(X)).all()

assert sorted(os.listdir (f'{path_results}/whole'))==['new_composition_result.pk', 'tr1_result.pk', 'tr2_result.pk']

composition1.set_split ('validation')
result1b = composition1 (X)
assert sorted(os.listdir (f'{path_results}/validation'))==['new_composition_result.pk', 'tr1_result.pk', 'tr2_result.pk']

remove_previous_results (path_results=f'{path_results}/whole')

resut_tr1_2 = composition2.tr1.data_io.load_result(split='validation')
resut_tr2_2 = composition2.tr2.data_io.load_result()

assert (resut_tr1_2==composition1.tr1(X)).all()
assert resut_tr2_2 is None

composition2.set_split('validation')
resut_tr1_2 = composition2.tr1.data_io.load_result()
resut_tr2_2 = composition2.tr2.data_io.load_result()

assert (resut_tr1_2==composition1.tr1(X)).all()
assert (resut_tr2_2==composition1.tr2(X)).all()

remove_previous_results (path_results=path_results)

#### Getting descendants

In [5]:
class Intermediate (MultiComponent):
    def __init__ (self, name=None, **kwargs):
        super().__init__ (name=name, **kwargs)
        self.first = Component (name='first_component', **kwargs)
        self.second = Component (name='second_component', **kwargs)

class Higher (MultiComponent):
    def __init__ (self, **kwargs):
        super().__init__ (**kwargs)
        self.first = Intermediate (name='first_intermediate', **kwargs)
        self.second = Intermediate (name='second_intermediate', **kwargs)
        self.gather_descendants(nick_name=False)

higher = Higher()

assert sorted(higher.obj.keys())==['first_component', 'first_intermediate', 'second_component', 'second_intermediate']

# check types
types = map(lambda x: type(x[1]), sorted(higher.obj.items()))
assert list(types)==[list, Intermediate, list, Intermediate]

sorted_list = list(sorted(higher.obj.items()))
types = map(type, sorted_list[0][1])
assert list(types)==[Component,Component]

sorted_list = list(sorted(higher.obj.items()))
types = map(type, sorted_list[2][1])
assert list(types)==[Component,Component]

sorted_keys=list(sorted(higher.cls.keys()))
assert sorted_keys == ['Component', 'Intermediate']

assert list(map(type,higher.cls[sorted_keys[0]]))==[Component, Component, Component, Component]

assert list(map(type,higher.cls[sorted_keys[1]]))==[Intermediate, Intermediate]


# ***********************
# recursive behaviour: higher.first
intermediate = higher.first
assert sorted(intermediate.obj.keys())==['first_component', 'second_component']

# check types
types = map(lambda x: type(x[1]), sorted(intermediate.obj.items()))
assert list(types)==[Component, Component]

sorted_keys=list(sorted(intermediate.cls.keys()))
assert sorted_keys==['Component']

assert list(map(type,intermediate.cls[sorted_keys[0]]))==[Component,Component]

# **********************************************
# recursive behaviour: higher.second
# **********************************************
intermediate = higher.second
assert sorted(intermediate.obj.keys())==['first_component', 'second_component']

# check types
types = map(lambda x: type(x[1]), sorted(intermediate.obj.items()))
assert list(types)==[Component, Component]

sorted_keys=list(sorted(intermediate.cls.keys()))
assert sorted_keys==['Component']

assert list(map(type,intermediate.cls[sorted_keys[0]]))==[Component,Component]

# **********************************************
# full hierarchical paths
# **********************************************
assert list(sorted(higher.full_cls.keys()))==['Component', 'Intermediate']
assert higher.full_cls['Intermediate']==['higher.first_intermediate', 'higher.second_intermediate']
assert higher.full_cls['Component']==['higher.first_intermediate.first_component',
  'higher.first_intermediate.second_component',
  'higher.second_intermediate.first_component',
  'higher.second_intermediate.second_component']

assert higher.first.full_cls['Component']==['higher.first_intermediate.first_component',
  'higher.first_intermediate.second_component']

assert list(sorted(higher.full_obj))==['first_component', 'first_intermediate', 'second_component', 'second_intermediate']

assert higher.full_obj['first_intermediate']=='higher.first_intermediate'

assert higher.full_obj['first_component']==['higher.first_intermediate.first_component',
  'higher.second_intermediate.first_component']

assert higher.full_obj['second_component']==['higher.first_intermediate.second_component',
  'higher.second_intermediate.second_component']

assert higher.full_obj['second_intermediate']=='higher.second_intermediate'

assert list(sorted(higher.second.full_obj))==['first_component', 'second_component']

assert higher.second.full_obj['first_component']=='higher.second_intermediate.first_component'

assert higher.second.full_obj['second_component']=='higher.second_intermediate.second_component'

assert higher.hierarchy_path=='higher'

assert higher.first.hierarchy_path=='higher.first_intermediate'

# with nick_names
higher.clear_descendants()
higher.gather_descendants(nick_name=True)

assert higher.full_cls['Component']==['higher.first.first',
 'higher.first.second',
 'higher.second.first',
 'higher.second.second']

assert higher.full_obj['first_intermediate']=='higher.first'
assert higher.full_obj['first_component']==['higher.first.first', 'higher.second.first']
assert higher.full_obj['second_component']==['higher.first.second', 'higher.second.second']
assert higher.full_obj['second_intermediate']=='higher.second'

In [6]:
# **********************************************
# second example
# **********************************************
class Intermediate (MultiComponent):
    def __init__ (self, name=None, **kwargs):
        super().__init__ (name=name, **kwargs)
        self.first = Component (name=f'{name}_first_component', **kwargs)
        self.second = Component (name=f'{name}_second_component', **kwargs)

class Higher (MultiComponent):
    def __init__ (self, **kwargs):
        super().__init__ (**kwargs)
        self.first = Intermediate (name='first_intermediate', **kwargs)
        self.second = Intermediate (name='second_intermediate', **kwargs)
        self.gather_descendants(nick_name=False)

higher = Higher()

assert sorted(higher.obj.keys())==['first_intermediate', 'first_intermediate_first_component', 'first_intermediate_second_component', 'second_intermediate', 'second_intermediate_first_component', 'second_intermediate_second_component']

# check types
types = map(lambda x: type(x[1]), sorted(higher.obj.items()))

assert list(types)==[Intermediate, Component, Component, Intermediate, Component, Component]

sorted_keys=list(sorted(higher.cls.keys()))
assert sorted_keys == ['Component', 'Intermediate']

assert list(map(type,higher.cls[sorted_keys[0]]))==[Component, Component, Component, Component]
assert list(map(type,higher.cls[sorted_keys[1]]))==[Intermediate, Intermediate]

# ***********************
# recursive behaviour: higher.first
intermediate = higher.first
assert sorted(intermediate.obj.keys())==['first_intermediate_first_component', 'first_intermediate_second_component']

# check types
types = map(lambda x: type(x[1]), sorted(intermediate.obj.items()))
assert list(types)==[Component, Component]

sorted_keys=list(sorted(intermediate.cls.keys()))
assert sorted_keys==['Component']

assert list(map(type,intermediate.cls[sorted_keys[0]]))==[Component,Component]

# ***********************
# recursive behaviour: higher.second
intermediate = higher.second
assert sorted(intermediate.obj.keys())==['second_intermediate_first_component', 'second_intermediate_second_component']

# check types
types = map(lambda x: type(x[1]), sorted(intermediate.obj.items()))
assert list(types)==[Component, Component]

sorted_keys=list(sorted(intermediate.cls.keys()))
assert sorted_keys==['Component']

assert list(map(type,intermediate.cls[sorted_keys[0]]))==[Component,Component]


# **********************************************
# full hierarchical paths
# **********************************************
assert list(sorted(higher.full_cls))==['Component', 'Intermediate']

assert higher.full_cls['Component']==['higher.first_intermediate.first_intermediate_first_component',
  'higher.first_intermediate.first_intermediate_second_component',
  'higher.second_intermediate.second_intermediate_first_component',
  'higher.second_intermediate.second_intermediate_second_component']

assert higher.full_cls['Intermediate']==['higher.first_intermediate', 'higher.second_intermediate']

assert list(higher.first.full_cls)==['Component']

assert higher.first.full_cls['Component']==['higher.first_intermediate.first_intermediate_first_component',
  'higher.first_intermediate.first_intermediate_second_component']

assert sorted(list(higher.full_obj))==['first_intermediate',
 'first_intermediate_first_component',
 'first_intermediate_second_component',
 'second_intermediate',
 'second_intermediate_first_component',
 'second_intermediate_second_component']

assert higher.full_obj['first_intermediate']=='higher.first_intermediate'

assert higher.full_obj['first_intermediate_first_component']=='higher.first_intermediate.first_intermediate_first_component'

assert higher.full_obj['first_intermediate_second_component']=='higher.first_intermediate.first_intermediate_second_component'

assert higher.full_obj['second_intermediate']=='higher.second_intermediate'

assert higher.full_obj['second_intermediate_first_component']=='higher.second_intermediate.second_intermediate_first_component'

assert higher.full_obj['second_intermediate_second_component']=='higher.second_intermediate.second_intermediate_second_component'

assert list(sorted(higher.second.full_obj))==['second_intermediate_first_component', 'second_intermediate_second_component']

assert higher.second.full_obj['second_intermediate_first_component']=='higher.second_intermediate.second_intermediate_first_component'

assert higher.second.full_obj['second_intermediate_second_component']=='higher.second_intermediate.second_intermediate_second_component'

assert higher.hierarchy_path=='higher'

assert higher.first.hierarchy_path=='higher.first_intermediate'

#### Passing parameters specific of hierarchy level

In [14]:
import pytest
class Intermediate (MultiComponent):
    def __init__ (self, name=None, z=6, h=10, x=3, **kwargs):
        super().__init__ (name=name, **kwargs)
        self.first = Component (name='first_component', **kwargs)
        self.second = Component (name='second_component', **kwargs)

class Higher (MultiComponent):
    def __init__ (self, x=2, y=3, **kwargs):
        super().__init__ (**kwargs)
        self.first = Intermediate (name='first_intermediate', **kwargs)
        self.second = Intermediate (name='second_intermediate', **kwargs)
        self.gather_descendants()

higher = Higher()

In [18]:
assert higher.x==2 and higher.y==3

In [16]:
with pytest.raises(AttributeError):
    print (higher.z)
with pytest.raises(AttributeError):
    print (higher.h)

In [21]:
assert higher.first.x==3 and higher.first.z==6 and higher.first.h==10

In [25]:
level=dict(
    x=10
)
higher = Higher (y=5, z=10, level_0=level)
assert higher.x==10 and higher.y==5 and higher.first.x==3

In [26]:
level=dict(
    x=10
)
higher = Higher (y=5, z=10, level_1=level)
assert higher.x==2 and higher.y==5 and higher.first.x==10

## Pipeline

In [ ]:
#export
class Pipeline (MultiComponent):
    """
    Pipeline composed of a list of components that run sequentially.
    
    During training, the components of the list are trained one after the other, 
    where one component is fed the result of transforming the data with the list 
    of components located before in the pipeline.
    
    The `Pipeline` class is a subclass of `SamplingComponent`, which itself is a 
    subclass of `Component`. This provides the functionality of `Component` 
    to any implemented pipeline, such as logging the messages, loading / saving the 
    results, and convert the data format so that it can work as part of other 
    pipelines with potentially other data formats.
    
    Being a subclass of `SamplingComponent`, the `transform` method 
    receives an input data  `X` that contains both data and labels. 
    
    Furthermore, the Pipeline constructor sets `separate_labels=False` by default,
    which means that the `fit` method also receives an input data `X` that contains 
    not only data but also labels. This is necessary because some of the components in 
    the pipeline might be of class `SamplingComponent`, and such components 
    need the input data `X` to contain labels when calling `transform` (and note that 
    this method is called when calling `fit` on a pipeline, since we do `fit_transform`
    on all the components except for the last one)
    """
    def __init__ (self, **kwargs):
        """Assigns attributes and calls parent constructor.

        Parameters
        ----------
        separate_labels: bool, optional
            whether or not the fit method receives the labels in a separate `y` vector 
            or in the same input `X`, as an additional variable. See description of 
            Pipeline class for more details.
        """
        
        super().__init__ (**kwargs)
        
    def _fit (self, X, y=None):
        """
        Fit components of the pipeline, given data X and labels y.
        
        By default, y will be None, and the labels are part of `X`, as a variable.
        """
        X = self._fit_apply_except_last (X, y)
        self.components[-1].fit (X, y)
    
    def _fit_apply (self, X, y=None, **kwargs):
        X = self._fit_apply_except_last (X, y, **kwargs)
        return self.components[-1].fit_apply (X, y, **kwargs)

    def _fit_apply_except_last (self, X, y, **kwargs):
        for component in self.components[:-1]:
            X = component.fit_apply (X, y, **kwargs)
        return X
    
    def _apply (self, X):
        """Transform data with components of pipeline, and predict labels with last component. 
        
        In the current implementation, we consider prediction a form of mapping, 
        and therefore a special type of transformation."""
        for component in self.components:
            X = component.transform (X)

        return X

### Usage examples / tests

#### Tests for `fit_apply` method

In [ ]:
# test `fit_apply` method
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.utils import Bunch
from block_types.core.block_types import PickleSaverComponent
from block_types.utils.utils import remove_previous_results

# Transform1: custom Transform
class Transform1 (Component):
    
    def __init__ (self, **kwargs):
        super().__init__ (**kwargs)
        self.estimator= Bunch(sum = 1)
        
    def _fit (self, X, y=None):
        self.estimator.sum = X.sum(axis=0)
    
    def _apply (self, x):
        return x*1000 + self.estimator.sum
    
class Transform2 (Component):
    
    def __init__ (self, **kwargs):
        super().__init__ (**kwargs)
        self.estimator= Bunch(maxim = 1)
        
    def _fit (self, X, y=None):
        self.estimator.maxim = X.max(axis=0)
    
    def _apply (self, x):
        return x*100 + self.estimator.maxim

class NewPipeline (Pipeline):
    
    def __init__ (self, **kwargs):
        super().__init__ (**kwargs)
        
        # custom transform
        self.tr1 = Transform1(**kwargs) 
        
        # slklearn transform
        self.tr2 = Transform2(**kwargs) 

pipeline = NewPipeline()
x = np.array([3,4,5])
r1 = pipeline.fit_apply (x.reshape(-1,1))
print (r1)

x1 = x * 1000 + sum(x)
x2 = x1 * 100 + max(x1)
assert (r1.ravel()==x2).all()

In [ ]:
class NewMulti (MultiComponent):
    
    def __init__ (self, **kwargs):
        super().__init__ (**kwargs)
        
        # custom transform
        self.tr1 = Transform1(**kwargs) 
        
        # slklearn transform
        self.tr2 = Transform2(**kwargs) 
        
    def _fit (self, X, y=None):
        self.tr1.fit (X)
        self.tr2.fit (X)
        
    def _apply (self, X, y=None):
        X1=self.tr1.apply (X)
        X2=self.tr2.apply (X)
        return X1+X2

new_multi = NewMulti()
r2 = new_multi.fit_apply (x)
print (r2)
x2b = 100 * x + max(x)
assert (r2.ravel()==(x1 + x2b)).all()

#### Adding new components to pipeline

In [ ]:
# test automatic creation of pipeline components
from sklearn.preprocessing import FunctionTransformer

# 1. by setting components as attributes:
class NewPipeline(Pipeline):
    def __init__ (self, **kwargs):
        super().__init__(**kwargs)
        self.tr1 = Component(FunctionTransformer (lambda x: x+1))
        self.tr2 = Component(FunctionTransformer (lambda x: x*2))
pipeline = NewPipeline()
result = pipeline.transform (3)
print (result)
assert result == 8

In [ ]:
#2. by using `set_components`
class NewPipeline(Pipeline):
    def __init__ (self, **kwargs):
        super().__init__(**kwargs)
        tr1 = Component(FunctionTransformer (lambda x: x+1))
        tr2 = Component(FunctionTransformer (lambda x: x*2))
        self.set_components (tr1, tr2)
        
        # the following transform is not added to the pipeline component list:
        self.tr3 = Component(FunctionTransformer (lambda x: x+1))
        
        # The reason is that once set_components is called, the component list 
        # is frozen and inmutable setting new components by attribute doesn't 
        # result in adding them to the component list
        
pipeline = NewPipeline()
result = pipeline.transform (3)

assert result == 8
assert len(pipeline.components) == 2
print (result, len(pipeline.components))

In [ ]:
#3. after calling `set_components()`, we can add new components with `add_component()`
class NewPipeline(Pipeline):
    def __init__ (self, **kwargs):
        super().__init__(**kwargs)
        tr1 = Component(FunctionTransformer (lambda x: x+1))
        tr2 = Component(FunctionTransformer (lambda x: x*2))
        self.set_components (tr1, tr2)
        
        tr3 = Component(FunctionTransformer (lambda x: x+2))
        self.add_component(tr3)
        
pipeline = NewPipeline()
result = pipeline.transform (3)

assert result == 10
assert len(pipeline.components) == 3
print (result, len(pipeline.components))

#### Tests for `load_estimator` method

In [ ]:
# test `load_estimator` method
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.utils import Bunch
from block_types.core.block_types import PickleSaverComponent
from block_types.utils.utils import remove_previous_results

# Transform1: custom Transform
class Transform1 (PickleSaverComponent):
    
    def __init__ (self, **kwargs):
        super().__init__ (**kwargs)
        self.estimator= Bunch(inv_c = 1)
        
    def _fit (self, X, y=None):
        self.estimator.inv_c = X.ravel()[0]
    
    def _apply (self, x):
        return x / self.estimator.inv_c

class NewPipeline (Pipeline):
    
    def __init__ (self, **kwargs):
        super().__init__ (**kwargs)
        
        # custom transform
        self.tr1 = Transform1(**kwargs) 
        
        # slklearn transform
        self.tr2 = PickleSaverComponent(StandardScaler(), **kwargs)
        
    def _fit (self, X, y=None):
        self.tr1.fit (X)
        self.tr2.fit (X)

# remove any previously stored 
path_results = 'pipeline_loading_saving'
remove_previous_results (path_results=path_results)

pipeline = NewPipeline(path_results=path_results, save_test_result=False)
pipeline.fit (np.array([3,4,5]).reshape(-1,1))
result1 = pipeline.transform (np.array([300,400,500]).reshape(-1,1))
print (pipeline.tr2.estimator.mean_)

del pipeline 
pipeline = NewPipeline(path_results=path_results, save_test_result=False)
pipeline.load_estimator ()
print (pipeline.tr2.estimator.mean_)
result2 = pipeline.transform (np.array([300,400,500]).reshape(-1,1))

np.testing.assert_array_equal (result1, result2)

# remove stored files resulting from running the current test
remove_previous_results (path_results=path_results)

### make_pipeline function

In [ ]:
# export
def make_pipeline(*components, cls=Pipeline, **kwargs):
    """Create `Pipeline` object of class `cls`, given `components` list."""
    pipeline = cls (**kwargs)
    pipeline.components = list(components)
    return pipeline

In [ ]:
tr1 = Component(FunctionTransformer (lambda x: x+1))
tr2 = Component(FunctionTransformer (lambda x: x*2))
pipeline = make_pipeline (tr1, tr2)
result = pipeline.transform (3)

print (result)
assert result == 8

### pipeline_factory

In [ ]:
# export
def pipeline_factory (pipeline_class, **kwargs):
    """Creates a pipeline object given its class `pipeline_class`
    
    Parameters
    ----------
    pipeline_class : class or str
        Name of the pipeline class used for creating the object. 
        This can be either of type string or class.
    """
    if type(pipeline_class) is str:
        Pipeline = eval(pipeline_class)
    elif type(pipeline_class) is type:
        Pipeline = pipeline_class
    else:
        raise ValueError (f'pipeline_class needs to be either string or class, we got {pipeline_class}')

    return Pipeline (**kwargs)

### PandasPipeline

In [ ]:
#export
class PandasPipeline (Pipeline):
    """
    Pipeline that saves results in parquet format, and preserves DataFrame format.
    
    See `Pipeline` class for an explanation of using `separate_labels=False`
    """
    def __init__ (self, 
                  data_converter=None,
                  data_io=None,
                  separate_labels=False,
                  **kwargs):
        if data_converter is None:
            data_converter = PandasConverter (separate_labels=separate_labels,
                                              **kwargs)
        if data_io is None:
            data_io = PandasIO (**kwargs)
        super().__init__ (self, 
                          data_converter=data_converter,
                          data_io=data_io,
                          **kwargs)

## ColumnTransformer

### ColumnSelector

In [ ]:
#export
class ColumnSelector (Component):
    def __init__ (self, 
                  columns=[],
                  **kwargs):
        super().__init__ (**kwargs)
        self.columns = columns
    
    def _apply (self, df):
        return df[self.columns]

In [ ]:
df = pd.DataFrame ({'x1': list(range(5)),
                    'x2': list(range(5,10)),
                    'x3': list(range(15,20)),
                    'x4': list(range(25,30))
                   })
dfr = ColumnSelector(columns=['x2','x4']).transform(df)
assert (dfr==df[['x2','x4']]).all().all()

### Concat

In [ ]:
#export
class Concat (Component):
    def __init__ (self, 
                  **kwargs):
        super().__init__ (**kwargs)
        
    def _apply (self, *dfs):
        return pd.concat(list(dfs), axis=1)

### ColumnTransformer

In [ ]:
# export
class _BaseColumnTransformer (MultiComponent):
    def __init__ (self, **kwargs):
        super().__init__ (**kwargs)
        self.concat = Concat (**kwargs)
    
    def _fit (self, df, y=None):
        for component in self.components:
            component.fit (df)
        return self
    
    def _apply (self, df):
        dfs = []
        for component in self.components:
            dfs.append (component.transform (df))
        df_result = self.concat.transform (*dfs)
        return df_result
    
class ColumnTransformer (_BaseColumnTransformer):
    def __init__ (self, *transformers, **kwargs):
        self.components = make_column_transformer_pipelines (*transformers, **kwargs)
        super().__init__ (**kwargs)

### Identity

In [ ]:
# export
class Identity (Component):
    def __init__ (self, **kwargs):
        super ().__init__ (**kwargs)
        
    def _apply (self, X):
        return X

### make_column_transformer

In [ ]:
# export
def make_column_transformer_pipelines (*transformers, **kwargs):
    pipelines = []
    for name, transformer, columns in transformers:
        if (type(transformer) is str) and transformer == 'passthrough':
            transformer = Identity (**kwargs)
        pipeline = make_pipeline(ColumnSelector(columns, **kwargs), 
                                 transformer, 
                                 name = name,
                                 **kwargs)
        pipelines.append (pipeline)
    
    return pipelines


def make_column_transformer (*transformers, **kwargs):
    transformers_with_name = []
    for transformer, columns in transformers:
        columns_name = ''.join([x[0] for x in columns])
        if len(columns_name) > 5:
            columns_name = columns_name[:5]
        if (type(transformer) is str) and transformer == 'passthrough':
            transformer_name = 'pass'
        elif hasattr(transformer, 'name'):
            transformer_name = transformer.name
        else:
            transformer_name = transformer.__class__.__name__
        name = f'{transformer_name}_{columns_name}'
        transformers_with_name.append ((name, transformer, columns))
    
    pipelines = make_column_transformer_pipelines (*transformers_with_name, **kwargs)
    column_transformer = _BaseColumnTransformer ()
    column_transformer.components = pipelines
    return column_transformer
    

### Tests / usage examples

In [ ]:
import pandas as pd
from sklearn.preprocessing import FunctionTransformer

df = pd.DataFrame ({'cont1': list(range(5)),
                    'cont2': list(range(5,10)),
                    'cont3': list(range(15,20)),
                    'cont4': list(range(25,30)),
                    'cat_1': list([1,2,3,2,1]),
                    'cat_2': list([0,1,1,0,0])
                    })

tr1 = Component(FunctionTransformer (lambda x: x+1), name='tr1')
tr2 = PandasComponent(FunctionTransformer (lambda x: x*2), transformed_columns=['cont2_bis','cat_1'], name='tr2')

column_transformer = make_column_transformer (
    (tr1, ['cont2', 'cont4']),
    (tr2, ['cont2', 'cat_1'])
)
dfr = column_transformer.transform(df)

# display and test
display(dfr)
assert (dfr[['cont2','cont4']] == tr1(df[['cont2','cont4']])).all().all()
assert (dfr[['cont2_bis','cat_1']] == tr2(df[['cont2','cat_1']])).all().all()
assert (dfr.columns == ['cont2','cont4', 'cont2_bis','cat_1']).all()

In [ ]:
column_transformer = make_column_transformer (
    (tr1, ['cont1', 'cont4']),
    ('passthrough', ['cont2', 'cat_1'])
)
dfr = column_transformer.transform(df)

# display and test
display(dfr)
assert (dfr[['cont1','cont4']] == tr1(df[['cont1','cont4']])).all().all()
assert (dfr[['cont2','cat_1']] == df[['cont2','cat_1']]).all().all()
assert (dfr.columns == ['cont1','cont4', 'cont2','cat_1']).all()

In [ ]:
class SumTimes100 (Component):
    def _fit (self, X, y=None):
        self.sum = X.sum(axis=0)
    def _apply (self, X):
        
        dfr = pd.DataFrame ({'c1_times100': self.sum.values[0]*100 + X.iloc[:,0].values,
                             'c2_times100': self.sum.values[1]*100 + X.iloc[:,1].values,
                             'c2_times1000': self.sum.values[1]*1000 + X.iloc[:,1].values})
        return dfr
        
tr1 = SumTimes100 ()
tr2 = PandasComponent(FunctionTransformer (lambda x: x*2), name='tr2')

column_transformer = make_column_transformer (
    (tr1, ['cont2', 'cont4']),
    (tr2, ['cont2', 'cat_1'])
)
dfr = column_transformer.fit_transform(df)

# display & test
display(dfr)
assert (dfr.columns == ['c1_times100','c2_times100', 'c2_times1000','cont2', 'cat_1']).all()
assert (dfr['c1_times100'] == sum(df.cont2)*100+df.cont2).all()
assert (dfr['c2_times100'] == sum(df.cont4)*100+df.cont4).all()
assert (dfr['c2_times1000'] == sum(df.cont4)*1000+df.cont4).all()

## MultiSplitComponent

In [ ]:
#export
class MultiSplitComponent (MultiComponent):
    def __init__ (self, 
                  component=None, 
                  fit_to = 'training',
                  fit_additional = [],
                  apply_to = ['training', 'validation', 'test'],
                  raise_error_if_split_doesnot_exist=False,
                  raise_warning_if_split_doesnot_exist=True,
                  **kwargs):
        super().__init__ (**kwargs)
        if component is not None:
            self.set_components (component)
            self.component = component
        
        self.fit_to = fit_to
        self.fit_additional = fit_additional
        self.apply_to = apply_to
        self.raise_error_if_split_doesnot_exist=raise_error_if_split_doesnot_exist
        self.raise_warning_if_split_doesnot_exist=raise_warning_if_split_doesnot_exist
    
    def _fit (self, X, y=None):
        if not isinstance(X, dict):
            X = {self.fit_to: X}
        component = self.components[0]
        additional_data = {}
        for split in self.fit_additional:
            if split not in ['validation', 'test']:
                raise ValueError (f'split {split} not valid')
            if split in X.keys():
                additional_data[f'{split}_data'] = X[split]
            else:
                self._issue_error_or_warning (split, X)
        
        component.fit(X[self.fit_to], y=y, split='training', **additional_data)
    
    def _issue_error_or_warning (self, split, X):
        message = f'split {split} not found in X keys ({X.keys()})'
        if self.raise_error_if_split_doesnot_exist:
            raise RuntimeError (message)
        elif self.raise_warning_if_split_doesnot_exist:
            warnings.warn (message)
    
    def _apply (self, X, apply_to = None, **kwargs):
        apply_to = self.apply_to if apply_to is None else apply_to
        apply_to = apply_to if isinstance(apply_to, list) else [apply_to]
        if not isinstance(X, dict):
            key = apply_to[0] if len(apply_to)==1 else 'test'
            X = {key: X}
            input_not_dict = True
        else:
            input_not_dict = False
            
        component = self.components[0]
        result = {}
        for split in apply_to:
            if split in X.keys():
                result[split] = component.apply (X[split], split=split, **kwargs)
            else:
                self._issue_error_or_warning (split, X)
        
        if input_not_dict:
            result = result[key]
        return result
        

### Tests / usage examples

In [ ]:
# example 1: apply transform on multiple splits
data = dict(training = np.array([1,2,3]).reshape(-1,1),
            validation = np.array([10,20,30]).reshape(-1,1),
            test = np.array([100,200,300]).reshape(-1,1)
            )

class Transform1 (Component):
    def __init__ (self, **kwargs):
        super().__init__ (**kwargs)
        self.estimator= Bunch(sum = 1)
        
    def _fit (self, X, y=None):
        self.estimator.sum = X.sum(axis=0)
    
    def _apply (self, x):
        return x*1000 + self.estimator.sum
    
multi_transform1 = MultiSplitComponent (component = Transform1())
result = multi_transform1.fit_transform (data)

assert type(result) is dict
assert result.keys() == data.keys()
for split in result.keys():
    assert (result[split]==sum(data['training'].ravel())+data[split]*1000).all()

In [ ]:
# example 2: fit method gets training, validation and test
# we apply the transform to only test
class Transform2 (Component):
    def __init__ (self, **kwargs):
        super().__init__ (**kwargs)
        self.estimator= Bunch(maxim = 1)
        
    def _fit (self, X, y=None, validation_data=None, test_data=None):
        self.estimator.maxim = X.max(axis=0)
        
        print (f'validation_data: {validation_data}')
        print (f'test_data: {test_data}')
        
        self.data = dict (validation=validation_data,
                          test=test_data)
    
    def _apply (self, x):
        return x*100 + self.estimator.maxim
        
tr2 = Transform2()
multi_transform2 = MultiSplitComponent (component=tr2,
                                        fit_additional = ['validation', 'test'])

# we apply the transform to only test data
result = multi_transform2.fit_transform (data, apply_to='test')

assert type(result) is dict
assert list(result.keys()) == ['test']
for split in result.keys():
    assert (result[split]==max(data['training'].ravel())+data[split]*100).all()

for split in ['validation', 'test']:
    assert (tr2.data[split] == data[split]).all()

In [ ]:
# test that we can chain transformations

result = multi_transform1.fit_transform (data)
result = multi_transform2.fit_transform (result, apply_to='test')

import pytest 

#check that we have no error if split does not exist
result = multi_transform1.fit_transform (data, apply_to=['training', 'validation'])
result = multi_transform2.fit_transform (result, apply_to=['test'])
assert len(result)==0

#check that we have an error if we set the flag `raise_error_if_split_doesnot_exist=True`
multi_transform2.raise_error_if_split_doesnot_exist = True
result = multi_transform1.fit_transform (data, apply_to=['training', 'validation'])
with pytest.raises (RuntimeError):
    result = multi_transform2.fit_transform (result, apply_to=['test'])

In [ ]:
# check loading / saving
from block_types.utils.utils import remove_previous_results
from block_types.core.utils import PickleIO

path_results = 'results_multi_split'

remove_previous_results (path_results=path_results)

tr = PickleSaverComponent (FunctionTransformer (lambda x: x*2),
                name='times2',
                path_results=path_results)

multi_transform = MultiSplitComponent (component=tr,
                                       apply_to = ['validation', 'test'],
                                       path_results = path_results,
                                       data_io=PickleIO (path_results = path_results))

result = multi_transform (data)

multi_transform2 = MultiSplitComponent (data_io=PickleIO (path_results = path_results))

result2 = multi_transform2.data_io.load_result ()

for k in result.keys():
    assert (result[k] == result2[k]).all()

assert result.keys()==result2.keys()

assert sorted(os.listdir(path_results))==['test', 'validation', 'whole']

assert (tr.data_io.load_result(split='test') == result['test']).all()

assert (tr.data_io.load_result(split='validation') == result['validation']).all()

assert os.listdir(f'{path_results}/validation')==['times2_result.pk']

assert os.listdir(f'{path_results}/test')==['times2_result.pk']

assert os.listdir(f'{path_results}/whole')==['multi_split_component_result.pk']

remove_previous_results (path_results=path_results)